In [1]:
import pycolmap

In [4]:
pycolmap.

Usage: hatch [OPTIONS] COMMAND [ARGS]...

   _   _       _       _
  | | | |     | |     | |
  | |_| | __ _| |_ ___| |__
  |  _  |/ _` | __/ __| '_ \
  | | | | (_| | || (__| | | |
  \_| |_/\__,_|\__\___|_| |_|

Options:
  -e, --env TEXT                  The name of the environment to use [env var:
                                  `HATCH_ENV`]
  -p, --project TEXT              The name of the project to work on [env var:
                                  `HATCH_PROJECT`]
  -v, --verbose                   Increase verbosity (can be used additively)
                                  [env var: `HATCH_VERBOSE`]
  -q, --quiet                     Decrease verbosity (can be used additively)
                                  [env var: `HATCH_QUIET`]
  --color / --no-color            Whether or not to display colored output
                                  (default is auto-detection) [env vars:
                                  `FORCE_COLOR`/`NO_COLOR`]
  --interactive / --no-interactive
     